# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
file_path = "../output_data/cities.csv"

df_all = pd.read_csv(file_path, encoding="utf-8")
df_all.dropna(how='any')
df_all.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [ ]:
# # Customize the size of the figure
# figure_layout = {
#     'width': '600px',
#     'height': '600px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }

fig = gmaps.figure(map_type="HYBRID")
heatmap_layer = gmaps.heatmap_layer(
    df_all[['Lat', 'Lng']], weights=df_all['Humidity'],
    max_intensity=50, point_radius=5, dissipating = False
)
fig.add_layer(heatmap_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
mask_temp = (df_all['Max Temp'] < 80) & (df_all['Max Temp'] > 70)
mask_wind = df_all["Wind Speed"] < 10
mask_cloud = df_all.Cloudiness == 0

df_new = df_all.loc[mask_temp & mask_wind & mask_cloud].reset_index(drop=True)
df_new = df_new.dropna(how="any")

df_new

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotels = []
addresses = []
location_codes = []
ratings = []

for _, row in df_new.iterrows():
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    print(response.url)
    # convert response to json
    places_data = response.json()
    
    try:
        name = places_data["results"][0]["name"]
        address = places_data["results"][0]["vicinity"]
        location_code = places_data["results"][0]["plus_code"]["compound_code"]
        rating = places_data["results"][0]["rating"]
    
        hotels.append(name)
        addresses.append(address)
        location_codes.append(location_code)
        ratings.append(rating)
    except:
        hotels.append("")
        addresses.append("")
        location_codes.append("")
        ratings.append(np.nan)
        
    time.sleep(1)
        
        
df_new["hotel"] = hotels
df_new["hotel_address"] = addresses
df_new["hotel_location_code"] = location_codes
df_new["rating"] = ratings

df_new["hotel_full_address"] = df_new["hotel_address"] + " " + df_new["hotel_location_code"]

In [ ]:
df_new

In [ ]:
hotel_df = df_new.dropna()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>Name</dt><dd>{rating}</dd>
<dt>Name</dt><dd>{hotel_full_address}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [ ]:
fig = gmaps.figure(map_type="HYBRID")
heatmap_layer = gmaps.heatmap_layer(
    df_all[['Lat', 'Lng']], weights=df_all['Humidity'],
    max_intensity=40, point_radius=5, dissipating = False
)
fig.add_layer(heatmap_layer)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig